# Data Cleaning and Modification

In [40]:
import pandas as pd

# Load raw data
tracks_df = pd.read_csv("dataset.csv")

# Drop old index column if it exists
if "Unnamed: 0" in tracks_df.columns:
    tracks_df = tracks_df.drop(columns=["Unnamed: 0"])

# Remove songs that share the same title (keep the first occurrence)
tracks_df = tracks_df.drop_duplicates(subset="track_name", keep="first")

# Discard any rows that contain missing values in *any* column
#tracks_df = tracks_df.dropna(axis=0, how="any")

# Convert explicit flag from boolean to integer (0/1)
tracks_df["explicit"] = tracks_df["explicit"].map({True: 1, False: 0})

# Create a helper field for searching: "Track name - artist1, artist2, ..."
def make_search_string(row):
    artist_list = [a.strip() for a in str(row["artists"]).split(";")]
    return f'{row["track_name"]} - {", ".join(artist_list)}'

tracks_df["search_string"] = tracks_df.apply(make_search_string, axis=1)

# Save cleaned version to disk
tracks_df.to_csv("dataset_clean.csv", index=False)

# Inspect the first few feature columns (adjust indices if needed)
tracks_df.columns.values[4:19]


array(['popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature'], dtype=object)

In [32]:
from pathlib import Path

def load_env_file(path="api.env"):
    """
    Very small .env-style loader.
    Returns a dict like {"LASTFM_API_KEY": "...", "LASTFM_API_SECRET": "..."}.
    """
    env_vars = {}
    env_path = Path(path)

    if not env_path.exists():
        raise FileNotFoundError(f"{path} not found in current folder")

    with env_path.open() as f:
        for line in f:
            line = line.strip()
            # skip empty lines and comments
            if not line or line.startswith("#"):
                continue
            if "=" not in line:
                continue
            key, value = line.split("=", 1)
            env_vars[key.strip()] = value.strip()

    return env_vars

env = load_env_file("api.env")
LASTFM_API_KEY = env.get("LASTFM_API_KEY")

if not LASTFM_API_KEY:
    raise ValueError("LASTFM_API_KEY not found or empty in api.env")


In [33]:
import os
import requests
from dotenv import load_dotenv
import time

LASTFM_ENDPOINT = "https://ws.audioscrobbler.com/2.0/"


def get_lastfm_metadata(artist_name: str, song_title: str):
    """
    Query Last.fm for basic track info.

    Returns a dict with playcount, listeners, duration and semicolon-separated tags,
    or None if nothing useful is returned.
    """
    params = {
        "method": "track.getInfo",
        "api_key": LASTFM_API_KEY,
        "artist": artist_name,
        "track": song_title,
        "autocorrect": 1,      # let Last.fm fix common typos
        "format": "json",
    }

    try:
        response = requests.get(LASTFM_ENDPOINT, params=params, timeout=5)
        response.raise_for_status()
        payload = response.json()

        if "error" in payload:
            print("Last.fm error:", payload)
            return None

        track_obj = payload.get("track", {})

        playcount = track_obj.get("playcount")
        listeners = track_obj.get("listeners")
        duration = track_obj.get("duration")

        playcount = int(playcount) if playcount is not None else None
        listeners = int(listeners) if listeners is not None else None
        duration = int(duration) if duration is not None else None

        raw_tags = track_obj.get("toptags", {}).get("tag", [])
        if isinstance(raw_tags, dict):
            raw_tags = [raw_tags]

        tag_names = []
        for t in raw_tags:
            name = t.get("name")
            if isinstance(name, str):
                tag_names.append(name.lower())

        tag_string = ";".join(tag_names) if tag_names else None

        return {
            "lfm_playcount": playcount,
            "lfm_listeners": listeners,
            "lfm_duration_ms": duration,
            "lfm_tags": tag_string,
        }

    except Exception as e:
        print("Request failed:", e)
        return None


In [34]:
test_artist = "Daft Punk"
test_track = "Get Lucky"

info = get_lastfm_metadata(test_artist, test_track)
print(info)


{'lfm_playcount': 1833302, 'lfm_listeners': 195540, 'lfm_duration_ms': 233000, 'lfm_tags': None}


In [41]:
import pandas as pd
from tqdm.auto import tqdm
import time

# Load cleaned data
tracks_df = pd.read_csv("dataset_clean.csv")

# Optional: sort by popularity if you have that column, so you enrich the “important” tracks first
if "popularity" in tracks_df.columns:
    tracks_df = tracks_df.sort_values("popularity", ascending=False).reset_index(drop=True)

# Limit how many tracks you hit the API with (for development)
N = 2000  # you can increase later
subset_idx = tracks_df.index[:N]


In [42]:
for col in ["lfm_playcount", "lfm_listeners", "lfm_duration_ms", "lfm_tags"]:
    tracks_df[col] = pd.NA


In [43]:
for idx in tqdm(subset_idx):
    row = tracks_df.loc[idx]
    song_title = row["track_name"]
    main_artist = str(row["artists"]).split(";")[0].strip()

    meta = get_lastfm_metadata(main_artist, song_title)

    if meta is not None:
        tracks_df.at[idx, "lfm_playcount"] = meta["lfm_playcount"]
        tracks_df.at[idx, "lfm_listeners"] = meta["lfm_listeners"]
        tracks_df.at[idx, "lfm_duration_ms"] = meta["lfm_duration_ms"]
        tracks_df.at[idx, "lfm_tags"] = meta["lfm_tags"]

    # Be polite to the API
    time.sleep(0.25)   # ~4 requests/second


  0%|          | 0/2000 [00:00<?, ?it/s]

Last.fm error: {'error': 6, 'message': 'Track not found', 'links': []}
Last.fm error: {'error': 6, 'message': 'Track not found', 'links': []}
Last.fm error: {'error': 6, 'message': 'Track not found', 'links': []}
Last.fm error: {'error': 6, 'message': 'Track not found', 'links': []}
Last.fm error: {'error': 6, 'message': 'Track not found', 'links': []}
Last.fm error: {'error': 6, 'message': 'Track not found', 'links': []}
